# Logistic Regression - Predicting Credit Card Default

### Data Set Information:

**This research aimed at the case of customers default payments in Taiwan**

### Features description:

- LIMIT_BAL: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
- SEX: Gender (1 = male; 2 = female). 
- EDUCATION: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- MARRIAGE: Marital status (1 = married; 2 = single; 3 = others). 
- AGE: Age (year). 
- PAY_0 - PAY_6: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: 0 = the repayment status in September, 2005; 1 = the repayment status in August, 2005; . . .; 6 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.
- BILL_AMT1-BILL_AMT6: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005. 
- PAY_AMT1-PAY_AMT6: Amount of previous payment (NT dollar).
- default payment next month: **positive class: default | negative class: pay**. Default means failed to pay credit bill next month

## Import libraries and models

In [16]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression # logistig regression model

## 1. Prepare data

In [17]:
default = pd.read_csv('./data/credit_card_default.csv', index_col="ID")
default.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [18]:
default = pd.read_csv('./data/credit_card_default.csv', index_col="ID")
# Cange column names to lower case
default.columns = default.columns.str.lower()
# For pay features if the <= 0 then it means it was not delayed. Set all negativ values to 0
pay_features = ['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6']
for p in pay_features:
    default.loc[default[p]<=0, p] = 0
# Rename target to shorter name
default.rename(columns={'default payment next month':'default'}, inplace=True) 

In [19]:
default.head()

,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,0,0,0,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,0,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,0,0,0,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [20]:
# Create dummy variables
# Base values: female, other_education, not_married
# == return True or False and astype('int') coverts to 1 for True and 0 for False
default['grad_school'] = (default['education'] == 1).astype('int')
default['university'] = (default['education'] == 2).astype('int')
default['high_school'] = (default['education'] == 3).astype('int')
default['male'] = (default['sex']==1).astype('int')
default['married'] = (default['marriage'] == 1).astype('int')
# Remove columns
default.drop(['education', 'sex', 'marriage'] , axis=1, inplace=True)
default.head()

,limit_bal,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,...,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default,grad_school,university,high_school,male,married
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,24,2,2,0,0,0,0,3913,3102,...,0,0,0,0,1,0,1,0,0,1
2,120000,26,0,2,0,0,0,2,2682,1725,...,1000,1000,0,2000,1,0,1,0,0,0
3,90000,34,0,0,0,0,0,0,29239,14027,...,1000,1000,1000,5000,0,0,1,0,0,0
4,50000,37,0,0,0,0,0,0,46990,48233,...,1200,1100,1069,1000,0,0,1,0,0,1
5,50000,57,0,0,0,0,0,0,8617,5670,...,10000,9000,689,679,0,0,1,0,1,1


### Building a Logistic Regression model with two features: pay_atm1 and Age
 
Positive class: Default(==1)
$$Pr(default=1 | X) = \frac{1}{1+exp(-Z)}$$
Where: 
$$Z = w_0 + w_1 PayAmt6 + w_2 Age$$

In [45]:
x = default[['pay_amt6','age']]
y = default['default']

## 3. Create an instance of a prediction model 

In [46]:
logreg = LogisticRegression()

## 4. Train model

In [47]:
logreg.fit(x, y)

LogisticRegression()

## 5. Evaluate model

In [48]:
# Print coefficients and intercept for the model
print(logreg.intercept_)
print(logreg.coef_)

[-1.33997627]
[[-1.41233276e-05  3.97680969e-03]]


$$Pr(default=1 | X) = \frac{1}{1+exp(-Z)}$$

Where: 

$$Z = -1.33998 - 0.00001 \times PayAmt6 + 0.00398 \times Age$$

In [51]:
# Create function for Z and probability
def Z(pay_amt, age):
    return logreg.intercept_[0] + logreg.coef_[0][0]*pay_amt + logreg.coef_[0][1]*age

def prob_default(pay_amt, age):
    z = Z(pay_amt, age) 
    return 1/(1 + np.exp(-z))

## Predicting probability of default

In [52]:
prob_default(pay_amt=0, age=40)

0.23488971532131808